In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras import models

In [ ]:
actions = ['星期二', '幾月幾號', '台北', '星期一', '星期三', '有', '什麼', '完了嗎', '將近', '父母', '一共', '買', '家裡', '房子', '銀行', '昨天', '認識', '見她', '今天', '久', '比較', '一', '手語', '你', '他們', '星期五', '是嗎', '誰', '還沒有', '星期六', '桃園', '朋友', '高鐵到', '吃飯', '會不會呢', '我問你', '生日', '我們兩個', '租', '棒', '孩子', '零', '星期天', '明天', '運動', '星期四', '捷運站', '一樣', '上課', '我', '年齡', '無', '天氣', '相見', '名字']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sequences = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_X_100.npy')
labels = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_Y_100.npy')

In [ ]:
np.array(sequences).shape 

(5500, 60, 165)

In [ ]:
np.array(labels).shape

(5500,)

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y.shape

(5500, 55)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

(275, 55)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, Bidirectional, BatchNormalization,ReLU,Activation
from tensorflow.keras.callbacks import TensorBoard


model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=True, activation='elu'), input_shape=(60,165)))
model.add(Bidirectional(LSTM(1024, return_sequences=True, activation='elu')))
model.add(Bidirectional(LSTM(512, return_sequences=False, activation='elu')))
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 60, 1024)          2777088   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 2048)          16785408  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              10489856  
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 55)                14135     
Total params: 30,722,615
Trainable params: 30,722,615
Non-trainable params: 0
____________________________________________

In [ ]:
#Bidirectional LSTM (100)
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/AI Python/Dataset/Dropout.h5', monitor='val_categorical_accuracy', mode="max", save_best_only=True, verbose=1)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode="auto", patience=3, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto')


model.fit(X_train, y_train,validation_split = 0.1,
              epochs=30,
              batch_size = 32,
              callbacks=[rlr,earlystopping])


Epoch 1/30
147/147 [==============================] - 3030s 21s/step - loss: nan - categorical_accuracy: 0.0317 - val_loss: nan - val_categorical_accuracy: 0.0172

Epoch 00001: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 2/30
147/147 [==============================] - 3248s 22s/step - loss: nan - categorical_accuracy: 0.0183 - val_loss: nan - val_categorical_accuracy: 0.0172

Epoch 00002: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 3/30
147/147 [==============================] - 3309s 23s/step - loss: nan - categorical_accuracy: 0.0183 - val_loss: nan - val_categorical_accuracy: 0.0172

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
Epoch 00003: early stopping


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'test_loss={test_loss}')
print(f'test_accuracy={test_accuracy}')

9/9 [==============================] - 35s 4s/step - loss: nan - categorical_accuracy: 0.0182
test_loss=nan
test_accuracy=0.0181818176060915
